# Github: initial fetch

In [1]:
import os
import json
import shutil
from pathlib import Path

import requests
import requests_cache

from nya.config import settings

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/michael/Documents/no-ur-awesome/py/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_84104/4184789187.py", line 9, in <module>
    from nya.config import settings
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/michael/Documents/no-ur-awesome/py/nya/config.py", line 28, in <module>
    settings = Settings()
               ^^^^^^^^^^
  File "pydantic/env_settings.py", line 39, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 342, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for Settings
github_token
  field required (type=value_error.missing)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/michael/Documents/no-ur-awe

In [ ]:
settings.dict()